In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
df =pd.read_csv('/kaggle/input/fake-bills/fake_bills.csv',sep=";")

In [50]:
first_column = df.pop(df.columns[0])
df.insert(len(df.columns), first_column.name, first_column)
df

,diagonal,height_left,height_right,margin_low,margin_up,length,is_genuine
0,171.81,104.86,104.95,4.52,2.89,112.83,True
1,171.46,103.36,103.66,3.77,2.99,113.09,True
2,172.69,104.48,103.50,4.40,2.94,113.16,True
3,171.36,103.91,103.94,3.62,3.01,113.51,True
4,171.73,104.28,103.46,4.04,3.48,112.54,True
...,...,...,...,...,...,...,...
1495,171.75,104.38,104.17,4.42,3.09,111.28,False
1496,172.19,104.63,104.44,5.27,3.37,110.97,False
1497,171.80,104.01,104.12,5.51,3.36,111.95,False
1498,172.06,104.28,104.06,5.17,3.46,112.25,False


In [51]:
def remove_outliers(df, column_name):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[column_name] < Q1 - 1.5 * IQR) | (df[column_name] > Q3 + 1.5 * IQR)]
    df.drop(outliers.index, inplace=True)
    return df

In [52]:
remove_outliers(df, 'diagonal')
remove_outliers(df, 'height_left')
remove_outliers(df, 'height_right')
remove_outliers(df, 'margin_low')
remove_outliers(df, 'margin_up')
remove_outliers(df, 'length')

,diagonal,height_left,height_right,margin_low,margin_up,length,is_genuine
1,171.46,103.36,103.66,3.77,2.99,113.09,True
2,172.69,104.48,103.50,4.40,2.94,113.16,True
3,171.36,103.91,103.94,3.62,3.01,113.51,True
4,171.73,104.28,103.46,4.04,3.48,112.54,True
5,172.17,103.74,104.08,4.42,2.95,112.81,True
...,...,...,...,...,...,...,...
1495,171.75,104.38,104.17,4.42,3.09,111.28,False
1496,172.19,104.63,104.44,5.27,3.37,110.97,False
1497,171.80,104.01,104.12,5.51,3.36,111.95,False
1498,172.06,104.28,104.06,5.17,3.46,112.25,False


In [53]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
df = imputer.fit_transform(df)

In [55]:
import numpy as np
from sklearn.model_selection import train_test_split
X = df[:, :-1]
y = df[:, -1]   
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('The shape of the train data is', y_train.shape[0])
print('The shape of the test data is', y_test.shape[0])

The shape of the train data is 1160
The shape of the test data is 291


In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': range(1, 21)}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(x_train, y_train)
best_k = grid_search.best_params_['n_neighbors']
best_accuracy = grid_search.best_score_

best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(x_train, y_train)
print('model score is: ' + str(round(best_knn.score(x_test, y_test)* 100, 0)) + '%')

model score is: 99.0%


In [73]:
random_sample = pd.DataFrame({
    'diagonal': np.random.uniform(170, 173, 2),
    'height_left': np.random.uniform(103, 105, 2),
    'height_right': np.random.uniform(103, 105, 2),
    'margin_low': np.random.uniform(3.5, 5.5, 2),
    'margin_up': np.random.uniform(2.9, 3.5, 2),
    'length': np.random.uniform(110, 114, 2)
})

predictions = best_knn.predict(random_sample)

predictions

array([1., 0.])